In [1]:
import pickle
from collections import Counter
from uuid import uuid4

from tqdm import tqdm

from datasets import load_dataset
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
train_dataset = load_dataset(
    "$HOME/gpt-code-clippy/code_clippy.py",
    data_dir="/home/shared/code-clippy-dataset/merged-data",
    streaming=True,
    split="train"
)

eval_dataset = load_dataset(
    "$HOME/gpt-code-clippy/code_clippy.py",
    data_dir="/home/shared/code-clippy-dataset/merged-data",
    streaming=True,
    split="validation"
)

Using the latest cached version of the module from /home/shpotes/.cache/huggingface/modules/datasets_modules/datasets/code_clippy/86b09b4a623c1c39753a8ad165e05757d9a97daf132ac71d3b6eb791e7da16dd (last modified on Fri Jul  9 22:06:59 2021) since it couldn't be found locally at $HOME/gpt-code-clippy/code_clippy.py/code_clippy.py or remotely (FileNotFoundError).
Using the latest cached version of the module from /home/shpotes/.cache/huggingface/modules/datasets_modules/datasets/code_clippy/86b09b4a623c1c39753a8ad165e05757d9a97daf132ac71d3b6eb791e7da16dd (last modified on Fri Jul  9 22:06:59 2021) since it couldn't be found locally at $HOME/gpt-code-clippy/code_clippy.py/code_clippy.py or remotely (FileNotFoundError).
Using custom data configuration default-668fb26707140662
Using the latest cached version of the module from /home/shpotes/.cache/huggingface/modules/datasets_modules/datasets/code_clippy/86b09b4a623c1c39753a8ad165e05757d9a97daf132ac71d3b6eb791e7da16dd (last modified on Fri Ju

In [3]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

In [4]:
def _get_stats(example):
    num_of_tokens = map(len, tokenizer(example["text"])['input_ids'])
    num_of_lines = map(lambda x: x.count("\n"), example["text"])
    file_name = map(lambda x: ".".join(x.split(".")[:-1]), example["file_name"])
    langs = list(map(lambda x: x.split(".")[-1], example["file_name"]))

    lang_map = {}
    acc_tok = []
    acc_lines = []
    acc_fnames = []

    for tok, lines, fname, lang in zip(num_of_tokens, num_of_lines, file_name, langs):
        if not lang in lang_map:
            lang_idx = len(acc_tok)
            lang_map[lang] = lang_idx

            acc_tok.append([tok])
            acc_lines.append([lines])
            acc_fnames.append([Counter({fname: 1})])
        else:
            lang_idx = lang_map[lang]

            acc_tok[lang_idx][0] += tok
            acc_lines[lang_idx][0] += lines
            acc_fnames[lang_idx][0].update({fname: 1})
    
    lang = [[k] for k, v in sorted(lang_map.items(), key=lambda item: item[1])]
    _id = [str(uuid4())] * len(lang)

    return {
        "ext": lang,
        "id": _id,
        "acc_num_of_tokens": acc_tok,
        "acc_num_of_lines": acc_lines,
        "acc_file_names": acc_fnames,
    }

In [5]:
 def collapse_metrics(per_language_dataset, target_file_name):
    num_of_tokens = {}
    num_of_lines = {}
    file_names = {}

    observed_lang = set()

    for row in tqdm(per_language_dataset):
        lang = row['ext'][0]
        if lang in observed_lang:
            num_of_tokens[lang] += row['acc_num_of_tokens'][0]
            num_of_lines[lang] += row['acc_num_of_lines'][0]
            file_names[lang].update(row['acc_file_names'][0])
        else:
            num_of_tokens[lang] = row['acc_num_of_tokens'][0]
            num_of_lines[lang] = row['acc_num_of_lines'][0]
            file_names[lang] = row['acc_file_names'][0]

    with open(target_file_name, 'wb') as buf:
        pickle.dump({
            'num_of_tokens': num_of_tokens,
            'num_of_lines': num_of_lines,
            'file_names': file_names,
        }, buf)

In [6]:
train_dataset = train_dataset.map(_get_stats, batched=True, batch_size=50_000)
eval_dataset = eval_dataset.map(_get_stats, batched=True, batch_size=50_000)

In [7]:
collapse_metrics(train_dataset, 'train_metrics.pkl')

0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2708 > 2048). Running this sequence through the model will result in indexing errors
61it [00:55,  1.36it/s]

In [ ]:
collapse_metrics(eval_dataset, 'eval_metrics.pkl')